# PDP / Forest Logging Detection

In [1]:
import ee
import geemap
import geopandas as gpd

## 1. Prezentacja danych ground truth

In [2]:
ee.Authenticate() 
ee.Initialize()

Unable to connect to VS Code server: Error in request.
Error: connect ENOENT /run/user/115586/vscode-ipc-166b8b18-2c6f-4bf3-a654-e6a86d6ca165.sock
    at PipeConnectWrap.afterConnect [as oncomplete] (node:net:1606:16) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'connect',
  address: '/run/user/115586/vscode-ipc-166b8b18-2c6f-4bf3-a654-e6a86d6ca165.sock'
}



Successfully saved authorization token.


In [3]:
def visualize_forest_change(year):
    if year < 2001 or year > 2023:
        raise ValueError("Podaj rok w zakresie od 2001 do 2023.")
    ee.Initialize()

    poland = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(
        ee.Filter.eq("ADM0_NAME", "Poland")
    )

    hansen_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")

    loss = hansen_data.select("loss")
    loss_year = hansen_data.select("lossyear")
    # Filtr strat lasu dla wybranego roku
    loss_in_year = loss.updateMask(loss_year.eq(year - 2000))
    loss_in_year_poland = loss_in_year.clip(poland)

    satellite_data_following_year = (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(f"{year + 1}-06-01", f"{year + 1}-08-31")
        .filterBounds(poland)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(poland)
    )

    satellite_data_previous_year = (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(f"{year - 1}-06-01", f"{year - 1}-08-31")
        .filterBounds(poland)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(poland)
    )

    # ustalenie obszaru zalesionego
    forest_cover = hansen_data.select("treecover2000").clip(poland).gte(25).selfMask()
    # .And(
    #     hansen_data \
    #         .select('lossyear') \
    #         .gt(year - 2000) \
    #         .Or(hansen_data.select('lossyear').eq(0))
    # )

    loss_viz = {"min": 0, "max": 1, "palette": ["red"]}
    satellite_viz = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.2}

    map = geemap.Map()
    map.centerObject(poland, 6)  # Przybliżenie na Polskę
    map.addLayer(satellite_data_following_year, satellite_viz, f"SAT {year + 1}")
    map.addLayer(satellite_data_previous_year, satellite_viz, f"SAT {year - 1}")
    map.addLayer(loss_in_year_poland, loss_viz, f"Straty lasu w {year}")
    map.addLayer(poland, {"color": "blue"}, "Granice Polski")
    map.addLayer(forest_cover, {"palette": ["green"]}, "Obszary leśne")

    return map

In [4]:
visualize_forest_change(2022)

/net/tscratch/people/plgknoculak/venv/lib/python3.11/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[52.108736119738694, 19.430639807941635], controls=(WidgetControl(options=['position', 'transparent…

In [5]:
def export_forest_loss_to_geojson(year, output_file):
    if year < 2001 or year > 2023:
        raise ValueError("Podaj rok w zakresie od 2001 do 2023.")

    ee.Initialize()

    poland = ee.FeatureCollection('FAO/GAUL/2015/level0') \
        .filter(ee.Filter.eq('ADM0_NAME', 'Poland'))
    poland_geometry = poland.geometry()

    hansen_data = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

    #Straty lasu
    loss = hansen_data.select('loss')
    loss_year = hansen_data.select('lossyear')


    loss_in_year = loss.updateMask(loss_year.eq(year - 2000))
    loss_in_year_poland = loss_in_year.clip(poland_geometry)


    loss_vector = loss_in_year_poland.reduceToVectors(
        geometry=poland_geometry,
        geometryType='polygon',
        reducer=ee.Reducer.countEvery(),
        scale=30,
        maxPixels=1e9, 
        bestEffort=True 
    )

    geemap.ee_export_vector(
        ee_object=loss_vector,
        filename=output_file
    )
export_forest_loss_to_geojson(2020, 'forest_loss_2020.geojson')


Generating URL ...
Please wait ...
Data downloaded to /net/people/plgrid/plgknoculak/pdp-forest-logging/forest_loss_2020.geojson


## 2. Klasyfikator

In [6]:
import ee
import geemap
import geopandas as gpd
from tqdm import tqdm

In [192]:
MIN_DEFORESTATION_YEAR = 2016
REFERENCE_YEAR = 2020
FOREST_COVER_THRESHOLD = 50
TRAINING_PATCH = ee.Geometry.Polygon([
    [17.5, 50.5],
    [17.5, 51.2],
    [19.3, 51.2],
    [19.3, 50.5]
])
TRAIN_TEST_SPLIT = 0.8
BOUNDING_BOX_BUFFER_METERS = 1000

In [193]:
def create_labeled_image(region_geometry, min_deforestation_year, reference_year, forest_cover_threshold):
    hansen_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").clip(region_geometry)
    treecover = hansen_data.select("treecover2000").gte(forest_cover_threshold)
    lossyear = hansen_data.select("lossyear")
    year_of_loss = lossyear.add(2000)
    
    class_forest = hansen_data.select("treecover2000").gt(forest_cover_threshold)
    
    # Class 2: Forest AND deforested between min_deforestation_year and reference_year
    class_recent_deforestation = (
        treecover
        .And(year_of_loss.gte(min_deforestation_year))
        .And(year_of_loss.lte(reference_year))
    ).selfMask().unmask(0)
    
    # Combine classes (mutually exclusive)
    labeled = (
        class_recent_deforestation.add(class_forest)
    ).rename("land_class")
    
    return labeled

In [194]:
# plot a map with "labeled" results from previous cell
labeled = create_labeled_image(
    TRAINING_PATCH, MIN_DEFORESTATION_YEAR, REFERENCE_YEAR, FOREST_COVER_THRESHOLD
)

# show a map
map = geemap.Map()

map.centerObject(TRAINING_PATCH, 12)
# map.addLayer(TRAINING_PATCH, {"color": "blue"}, "Training Patch")
map.addLayer(labeled, {"min": 0, "max": 2, "palette": ["black", "green", "red"]})
map

Map(center=[50.85258536025442, 18.399999999999924], controls=(WidgetControl(options=['position', 'transparent_…

In [195]:
def extract_training_data(
    region_geometry,
    min_deforestation_year,
    reference_year,
    forest_cover_threshold,
    train_test_split,
):
    labeled = create_labeled_image(
        region_geometry, min_deforestation_year, reference_year, forest_cover_threshold
    )
    sat_image = (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(f"{reference_year + 1}-06-01", f"{reference_year + 1}-08-31")
        .filterBounds(region_geometry)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(region_geometry)
    )
    merged = sat_image.addBands(labeled)
    points_all = merged.sample(
        region=region_geometry,
        scale=30,
        numPixels=250000,
        seed=42,
        tileScale=2,
        geometries=True,
    )

    none_points = points_all.filter(ee.Filter.eq("land_class", 0))
    forest_points = points_all.filter(ee.Filter.eq("land_class", 1))
    deforest_points = points_all.filter(ee.Filter.eq("land_class", 2))

    none_count = none_points.size().getInfo()
    forest_count = forest_points.size().getInfo()
    deforest_count = deforest_points.size().getInfo()

    min_count = min(none_count, forest_count, deforest_count)

    none_points_limited = none_points.limit(min_count)
    forest_points_limited = forest_points.limit(min_count)
    deforest_points_limited = deforest_points.limit(min_count)

    balanced_points = none_points_limited.merge(forest_points_limited).merge(
        deforest_points_limited
    )

    print("Balanced points count:", min_count, "(per class)")

    points_with_random = balanced_points.randomColumn("random_value", seed=42)
    train_points = points_with_random.filter(
        ee.Filter.lt("random_value", train_test_split)
    )
    test_points = points_with_random.filter(
        ee.Filter.gte("random_value", train_test_split)
    )

    return sat_image, labeled, train_points, test_points

In [221]:
def show_training_samples(sat_image, train_points, test_points):
    all_points = ee.FeatureCollection(train_points).merge(test_points)
    color_mapped = all_points.map(
        lambda f: f.set("style", {
            "color": ee.Algorithms.If(
                f.getNumber("land_class").eq(0),
                "grey",
                ee.Algorithms.If(
                    f.getNumber("land_class").eq(1),
                    "green",
                    "orange"
                )
            ),
            "pointSize": 5
        })
    )
    m = geemap.Map()
    m.centerObject(all_points.geometry(), 8)
    rgb_viz = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.2}
    m.addLayer(sat_image, rgb_viz, "Sentinel")
    m.addLayer(color_mapped.style(**{"styleProperty": "style"}), {}, "Train/Test Points")
    return m


In [217]:
import pickle
import json


def train_classifier(train_points, test_points, bands, classifier=None, disable_train_val=False, train_suite_name="final"):
    print("Trenuję klasyfikator...")
    if classifier is None:
        classifier = ee.Classifier.smileRandomForest(numberOfTrees=50)
    
    classifier = classifier.train(
        features=train_points, classProperty="land_class", inputProperties=bands
    )

    classifier_str = "_".join([f"{k}_{v}" for k, v in classifier.getInfo()['classifier'].items()])
    classifier_str = classifier_str.replace(".", "_")
    
    print(f"Zapisuje klasyfikator jako: {classifier_str}")
    
    print("Obliczam metryki treningowe i testowe...")
    test_pred = test_points.classify(classifier)
    
    conf_matrix_test_info = test_pred.errorMatrix("land_class", "classification").getInfo()
    
    # Obliczanie dokładności bezpośrednio z macierzy błędów
    def compute_accuracy(conf_matrix):
        correct = sum(conf_matrix[i][i] for i in range(len(conf_matrix)))
        total = sum(sum(row) for row in conf_matrix)
        return correct / total if total > 0 else 0

    
    accuracy_test = compute_accuracy(conf_matrix_test_info)
    
    
    if not disable_train_val:
        train_pred = train_points.classify(classifier)
        conf_matrix_train_info = train_pred.errorMatrix("land_class", "classification").getInfo()
        accuracy_train = compute_accuracy(conf_matrix_train_info)
        print("Macierz błędów (train):", conf_matrix_train_info)
        print("Dokładność (train):", accuracy_train)
    print("Macierz błędów (test):", conf_matrix_test_info)
    print("Dokładność (test):", accuracy_test)
    
    # Zapis wyników
    result = {
        "conf_matrix_train": conf_matrix_train_info if not disable_train_val else None,
        "accuracy_train": accuracy_train if not disable_train_val else None,
        "conf_matrix_test": conf_matrix_test_info,
        "accuracy_test": accuracy_test
    }
    
    with open(f"results_{train_suite_name}/{classifier_str}.json", "w") as f:
        json.dump(result, f)
    
    with open(f"models_{train_suite_name}/{classifier_str}.pkl", "wb") as f:
        pickle.dump(classifier, f)
    
    return classifier


In [198]:
def classify_and_display_results(classifier, bands, bounding_box_buffer_m, reference_year, forest_cover_threshold):
    print("Wczytuję dane i generuję klasyfikację dla całej Polski...")
    poland = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq("ADM0_NAME", "Poland")).first()
    bbox = poland.geometry().bounds().buffer(bounding_box_buffer_m)
    poland_clipped = ee.Feature(poland).geometry().intersection(bbox)
    hansen_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").clip(poland_clipped)
    sat_image = (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(f"{reference_year + 1}-06-01", f"{reference_year + 1}-08-31")
        .filterBounds(poland_clipped)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(poland_clipped)
    )
    classified = sat_image.select(bands).classify(classifier).rename("classified")
    m = geemap.Map()
    m.centerObject(ee.FeatureCollection([poland]).geometry(), 6)
    m.add_basemap("OpenStreetMap")
    rgb_viz = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.2}
    class_viz = {"min": 0, "max": 2, "palette": ["grey", "green", "orange"]}
    m.addLayer(sat_image, rgb_viz, "Obraz Sentinel")
    m.addLayer(classified, class_viz, "Klasyfikacja")
    m.addLayer(hansen_data.select("lossyear"), {"min": 0, "max": 23, "palette": ["black", "yellow", "red"]}, "Hansen Loss Year")
    forest_mask = hansen_data.select("treecover2000").gte(forest_cover_threshold).selfMask()
    m.addLayer(forest_mask, {"palette": ["green"]}, "Hansen Forest 2000")
    return m


In [199]:
poland_fc = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq("ADM0_NAME", "Poland"))
region_for_training = poland_fc.geometry().intersection(TRAINING_PATCH)
sat_image, labeled_image, train_points, test_points = extract_training_data(
    region_for_training,
    MIN_DEFORESTATION_YEAR,
    REFERENCE_YEAR,
    FOREST_COVER_THRESHOLD,
    TRAIN_TEST_SPLIT
)
hansen_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").clip(region_for_training)


Balanced points count: 4682 (per class)


In [218]:
train_points = train_points.randomColumn()
test_points = test_points.randomColumn()
train_points_mini = train_points.limit(150, "random")
test_points_mini = test_points.limit(150, "random")

In [219]:
map_samples = show_training_samples(sat_image, train_points_mini, test_points_mini)
map_samples

In [ ]:
print("Trenuję model...")
bands = ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]

classifiers = {
    #"Random Forest n=50": ee.Classifier.smileRandomForest(numberOfTrees=50),
    #"Random Forest n=100": ee.Classifier.smileRandomForest(numberOfTrees=100),
    #"Random Forest n=200": ee.Classifier.smileRandomForest(numberOfTrees=200),
    #"Random Forest n=500": ee.Classifier.smileRandomForest(numberOfTrees=500),
    #"Gradient Tree Boosting n=50": ee.Classifier.smileGradientTreeBoost(
    #     numberOfTrees=50
    # ),
    #"Gradient Tree Boosting n=100": ee.Classifier.smileGradientTreeBoost(
    #    numberOfTrees=100
    #),
    #"Gradient Tree Boosting n=200": ee.Classifier.smileGradientTreeBoost(
    #    numberOfTrees=200
    #),
    #"Gradient Tree Boosting n=500": ee.Classifier.smileGradientTreeBoost(
    #    numberOfTrees=500
    #),
    #"SVM": ee.Classifier.libsvm(),
    "k-NN k=3": ee.Classifier.smileKNN(k=3),
    "k-NN k=5": ee.Classifier.smileKNN(k=5),
    "k-NN k=7": ee.Classifier.smileKNN(k=7),
}

for classifier_name in classifiers.keys():
    print(f"==================== {classifier_name} ====================")
    classifiers[classifier_name] = train_classifier(
        train_points, test_points, bands, classifiers[classifier_name]
    )
    print("============================================================")

In [108]:
print("Klasyfikuję i wyświetlam wyniki dla całej Polski...")
classification_map = classify_and_display_results(
    classifiers["Random Forest n=200"],
    bands,
    BOUNDING_BOX_BUFFER_METERS,
    REFERENCE_YEAR,
    FOREST_COVER_THRESHOLD
)
classification_map


Klasyfikuję i wyświetlam wyniki dla całej Polski...
Wczytuję dane i generuję klasyfikację dla całej Polski...


Map(center=[52.108736119738694, 19.430639807941635], controls=(WidgetControl(options=['position', 'transparent…

In [76]:
def lookup(source_hist, target_hist):
    """Creates a lookup table to make a source histogram match a target histogram.

    Args:
        source_hist: The histogram to modify. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.
        target_hist: The histogram to match to. Expects the Nx2 array format produced by ee.Reducer.autoHistogram.

    Returns:
        A dictionary with 'x' and 'y' properties that respectively represent the x and y
        array inputs to the ee.Image.interpolate function.
    """

    # Split the histograms by column and normalize the counts.
    source_values = source_hist.slice(1, 0, 1).project([0])
    source_counts = source_hist.slice(1, 1, 2).project([0])
    source_counts = source_counts.divide(source_counts.get([-1]))

    target_values = target_hist.slice(1, 0, 1).project([0])
    target_counts = target_hist.slice(1, 1, 2).project([0])
    target_counts = target_counts.divide(target_counts.get([-1]))

    # Find first position in target where targetCount >= srcCount[i], for each i.
    def make_lookup(n):
        return target_values.get(target_counts.gte(n).argmax())

    lookup = source_counts.toList().map(make_lookup)

    return {"x": source_values.toList(), "y": lookup}


def histogram_match(source_img, target_img, geometry, bands):

    args = {
        "reducer": ee.Reducer.autoHistogram(maxBuckets=256, cumulative=True),
        "geometry": geometry,
        "scale": 1,  # Need to specify a scale, but it doesn't matter what it is because bestEffort is true.
        "maxPixels": 65536 * 4 - 1,
        "bestEffort": True,
    }

    # Only use pixels in target that have a value in source (inside the footprint and unmasked).
    source = source_img.reduceRegion(**args)
    target = target_img.updateMask(source_img.mask()).reduceRegion(**args)

    interpolated_bands = []
    for band in bands:
        interpolated_band = source_img.select([band]).interpolate(
            **lookup(source.getArray(band), target.getArray(band))
        )
        interpolated_bands.append(interpolated_band)
    
    return ee.Image(ee.Image.cat(*interpolated_bands).copyProperties(source_img, ["system:time_start"]))


def find_closest(target_image, image_col, days):
    """Filter images in a collection by date proximity and spatial intersection to a target image.

    Args:
        target_image: An ee.Image whose observation date is used to find near-date images in
          the provided image_col image collection. It must have a 'system:time_start' property.
        image_col: An ee.ImageCollection to filter by date proximity and spatial intersection
          to the target_image. Each image in the collection must have a 'system:time_start'
          property.
        days: A number that defines the maximum number of days difference allowed between
          the target_image and images in the image_col.

    Returns:
        An ee.ImageCollection that has been filtered to include those images that are within the
          given date proximity to target_image and intersect it spatially.
    """

    # Compute the timespan for N days (in milliseconds).
    range = ee.Number(days).multiply(1000 * 60 * 60 * 24)

    filter = ee.Filter.And(
        ee.Filter.maxDifference(range, "system:time_start", None, "system:time_start"),
        ee.Filter.intersects(".geo", None, ".geo"),
    )

    closest = ee.Join.saveAll("matches", "measure").apply(
        ee.ImageCollection([target_image]), image_col, filter
    )

    return ee.ImageCollection(ee.List(closest.first().get("matches")))

In [79]:
def classify_and_subtract(classifier, bands, bounding_box_buffer_m, year_from, year_to, forest_cover_threshold):
    print("Wczytuję dane i generuję klasyfikację dla całej Polski...")
    poland = ee.FeatureCollection("FAO/GAUL/2015/level0").filter(ee.Filter.eq("ADM0_NAME", "Poland")).first()
    bbox = poland.geometry().bounds().buffer(bounding_box_buffer_m)
    poland_clipped = ee.Feature(poland).geometry().intersection(bbox)
    hansen_data = ee.Image("UMD/hansen/global_forest_change_2023_v1_11").clip(poland_clipped)
    
    ref_sat_image = (
        ee.ImageCollection("COPERNICUS/S2")
        .filterDate(f"2020-06-01", f"2020-08-31")
        .filterBounds(poland_clipped)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .median()
        .clip(poland_clipped)
    )    


    sat_images = []
    for yr in (year_from, year_to):
        sat_image = (
            ee.ImageCollection("COPERNICUS/S2")
            .filterDate(f"{yr}-06-01", f"{yr}-08-31")
            .filterBounds(poland_clipped)
            .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
            .median()
            .clip(poland_clipped)
        )
        sat_images.append(sat_image)
    
    matched_sat_images = []
    for i in range(2):
        matched_sat_images.append(
            histogram_match(
                sat_images[i],
                ref_sat_image,
                poland_clipped,
                bands
            )
        )


    classified_from = matched_sat_images[0].select(bands).classify(classifier).rename("classified start")
    classified_to = matched_sat_images[1].select(bands).classify(classifier).rename("classified start")
    
    m = geemap.Map()
    m.centerObject(ee.FeatureCollection([poland]).geometry(), 6)
    rgb_viz = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.2}
    class_viz = {"min": 0, "max": 2, "palette": ["grey", "green", "orange"]}
    m.addLayer(sat_images[0], rgb_viz, f"Sentinel {year_from}")
    m.addLayer(sat_images[1], rgb_viz, f"Sentinel {year_to}")
    m.addLayer(matched_sat_images[0], rgb_viz, f"Sentinel Matched {year_from}")
    m.addLayer(matched_sat_images[1], rgb_viz, f"Sentinel Matched {year_to}")
    m.addLayer(ref_sat_image, rgb_viz, "Sentinel 2020 (Reference)")
    
    m.addLayer(classified_from, class_viz, "Klasyfikacja from")
    m.addLayer(classified_to, class_viz, "Klasyfikacja to")
    #m.addLayer(hansen_data.select("lossyear"), {"min": 0, "max": 23, "palette": ["black", "yellow", "red"]}, "Hansen Loss Year")
    #forest_mask = hansen_data.select("treecover2000").gte(forest_cover_threshold).selfMask()
    #m.addLayer(forest_mask, {"palette": ["green"]}, "Hansen Forest 2000")
    return m

classify_and_subtract(classifiers["Random Forest n=200"], bands, BOUNDING_BOX_BUFFER_METERS, 2019, 2022, FOREST_COVER_THRESHOLD)

Wczytuję dane i generuję klasyfikację dla całej Polski...


Map(center=[52.108736119738694, 19.430639807941635], controls=(WidgetControl(options=['position', 'transparent…